In [ ]:
import pandas as pd
import geopandas as gpd
from os import path, makedirs
from unidecode import unidecode

In [ ]:
_url_dist_mun = (
    "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:distrito_municipal&outputFormat=application/json"
)
_url_subs = (
    "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:subprefeitura&outputFormat=application/json"
)
_url_dist_censo = (
    "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/gpkg/UF/SP/SP_distritos_CD2022.gpkg"
)

### Indicador domicilios não ocupados (censo 2022)

In [ ]:
if "dist_mun" not in globals():
    _url_dist_mun = "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:distrito_municipal&outputFormat=application/json"
    dist_mun = gpd.read_file(_url_dist_mun)

In [ ]:

if "subs" not in globals():
    _url_subs = "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:subprefeitura&outputFormat=application/json"
    subs = gpd.read_file(_url_subs)

In [ ]:

if "dist_censo" not in globals():
    _url_dist_censo = "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/gpkg/UF/SP/SP_distritos_CD2022.gpkg"
    dist_censo = gpd.read_file(_url_dist_censo)

In [ ]:
dist_censo = dist_censo[dist_censo["CD_MUN"] == "3550308"].copy()
dist_censo["nm_distrito_municipal"] = dist_censo["NM_DIST"].apply(unidecode).str.upper()

In [ ]:
dist_mun

In [ ]:
dist_censo

In [ ]:
subs

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [ ]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

for nome, df in [
    ('Distrito', dist_mun),
    ('Setor-censitario', dist_censo),
    ('Subprefeituras', subs),
]:
    filename = path.join(output_dir, nome)
    df.to_csv(f'{filename}.csv', sep=';', decimal=',', encoding='utf8', index=False)
